In [1]:
import numpy as np
from collections import defaultdict
from qiskit import Aer
from qiskit.visualization import plot_histogram
from qiskit.utils import QuantumInstance
from qiskit.algorithms import Grover, AmplificationProblem
from qiskit.circuit.library import PhaseOracle
from qiskit.test.mock import FakeQuito
from qiskit.compiler import transpile
from qiskit import assemble
from qiskit import IBMQ
from qiskit import execute
from qiskit.tools import job_monitor
import matplotlib.pyplot as plt
IBMQ.save_account('', overwrite=True)
IBMQ.load_account()
provider = IBMQ.get_provider(hub = 'ibm-q')
device = provider.get_backend('ibmq_quito')
quito = FakeQuito()
circuit_list = []
verification_list = []

In [2]:
class Verifier():
    """Create an object that can be used to check whether
    an assignment satisfies a DIMACS file.
        Args:
            dimacs_file (str): path to the DIMACS file
    """
    def __init__(self, dimacs_file):
        with open(dimacs_file, 'r') as f:
            self.dimacs = f.read()

    def is_correct(self, guess):
        """Verifies a SAT solution against this object's
        DIMACS file.
            Args:
                guess (str): Assignment to be verified.
                             Must be string of 1s and 0s.
            Returns:
                bool: True if `guess` satisfies the
                           problem. False otherwise.
        """
        # Convert characters to bools & reverse
        guess = [bool(int(x)) for x in guess][::-1]
        for line in self.dimacs.split('\n'):
            line = line.strip(' 0')
            clause_eval = False
            for literal in line.split(' '):
                if literal in ['p', 'c']:
                    # line is not a clause
                    clause_eval = True
                    break
                if '-' in literal:
                    literal = literal.strip('-')
                    lit_eval = not guess[int(literal)-1]
                else:
                    lit_eval = guess[int(literal)-1]
                clause_eval |= lit_eval
            if clause_eval is False:
                return False
        return True

In [ ]:
N=4
opt_lvl = 3
circuit_list.clear()
verification_list.clear()
for andIter in range(N,N*3+1):#the ands
    for orIter in range(andIter*2,(N-1)*(andIter+1)+1): #the ors
        for notIter in range(3, 9, 2): #not composition
            thenotcomp = notIter*0.1
            filename = "dimacs/demosatN"+str(N)+"Ands"+str(andIter)+"Ors"+str(orIter)+"NotComp"+format(thenotcomp, '.1f')+".dimacs"
            with open(filename, 'r') as f:
                dimacs = f.read()
            #print(dimacs)
            oracle = PhaseOracle.from_dimacs_file(filename)
            oracle.draw()
            v = Verifier(filename) 
            verification_list.append(v)
            #v.is_correct('101')
            # Configure backend
            backend = Aer.get_backend('aer_simulator')
            quantum_instance = QuantumInstance(backend, shots=1024)

            # Create a new problem from the phase oracle and the
            # verification function
            problem = AmplificationProblem(oracle=oracle, is_good_state=v.is_correct)

            # Use Grover's algorithm to solve the problem
            grover = Grover(quantum_instance=quantum_instance)
            result = grover.amplify(problem)
            '''
            result.top_measurement


            dd = defaultdict(list)

            for d in result.circuit_results : # you can list as many input dicts as you want here
                for key, value in d.items():
                    dd[key].append(value)
            for key in dd:
                dd[key] = sum(dd[key])
            print(dd)
            '''

            #print(result.circuit_results)
            #plot_histogram(result.circuit_results[-1])

            # transpile the circuit for ibmq_belem
            #print(max(result.iterations))
            qc = grover.construct_circuit(problem, max(result.iterations))
            qc.measure_all()
            grover_compiled = transpile(qc, backend=quito, optimization_level=opt_lvl)

            #print('gates = ', grover_compiled.count_ops())
            #print('depth = ', grover_compiled.depth())

            counts = quito.run(grover_compiled).result().get_counts()
            #plot_histogram(counts)

            qc_transpiled = transpile(qc, optimization_level=opt_lvl)
            print(qc_transpiled.depth())
            #qc_transpiled.draw(output='mpl')
            #plt.show()
            #print(qc_transpiled.decompose())
            circuit_list.append(qc_transpiled)

99
84
79
108
78
64
73
179
66
162
176
166
96
204
168
98
195
156
125
192
164
198
194
202
82
82
67
52
62
66
200
63
109
116
57
200
176
105
186
192
187
135
126
79
218
94
64
224
150
126
121
33
47
101
98
70
108
83
65
35
70
52
109
64
114
108
78
84
38
65
82
62
214
50


C:\Users\Ahmad\anaconda3\envs\qc_env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized [<qiskit.dagcircuit.dagnode.DAGNode object at 0x000002A1D03529A0>, <qiskit.dagcircuit.dagnode.DAGNode object at 0x000002A1D0352520>] and got global phase: π/2
      ┌────────┐┌────┐┌────────┐
qr_0: ┤ Rz(-π) ├┤ √X ├┤ Rz(-π) ├
      └────────┘└────┘└────────┘, but the original was native and the new value is longer.  This indicates an efficiency bug in synthesis.  Please report it by opening an issue here: https://github.com/Qiskit/qiskit-terra/issues/new/choose
  new_dag = pass_.run(dag)
C:\Users\Ahmad\anaconda3\envs\qc_env\lib\site-packages\qiskit\transpiler\runningpassmanager.py:166: UserWarning: Resynthesized [<qiskit.dagcircuit.dagnode.DAGNode object at 0x000002A17BC8A040>, <qiskit.dagcircuit.dagnode.DAGNode object at 0x000002A1D0193880>] and got global phase: π/2
      ┌────────┐┌────┐┌────────┐
qr_0: ┤ Rz(-π) ├┤ √X ├┤ Rz(-π) ├
      └────────┘└────┘└──────

149
98
200
190
188


In [ ]:
#job = device.retrieve_job("6310d41dc815296574e0f369")
#print(job)

In [ ]:
job = execute(circuit_list,backend = device,shots = 1024)
print(job.job_id())

In [ ]:
job_monitor(job)
device_result = job.result()
#plot_histogram(device_result.get_counts())

In [ ]:
#v = Verifier('dimacs/demosatN3Ands7Ors16NotComp0.7.dimacs')
#v.is_correct('111')
job_counter = 0
N=3
for andIter in range(N,N*3+1):#the ands
    for orIter in range(andIter*2,(N-1)*(andIter+1)+1): #the ors
        for notIter in range(3, 9, 2): #not composition
            thenotcomp = notIter*0.1
            filename = "dimacs/demosatN"+str(N)+"Ands"+str(andIter)+"Ors"+str(orIter)+"NotComp"+format(thenotcomp, '.1f')+".dimacs"
            v = Verifier(filename)
            #print(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get))
            print(v.is_correct(max(device_result.get_counts()[job_counter], key=device_result.get_counts()[job_counter].get)))
            job_counter +=1

In [ ]:
print(len(device_result.get_counts()))